In [ ]:
from pdstools.explanations import GradientBoostGlobalExplanations, ExplanationsExplorer, ExplanationsDataLoader

import datetime
import polars as pl

## Aggregate data exported from infinity

If this step has not already been done, you need to aggregate the explanation data exported from infinity. Unless otherwise specified, the resulting aggrgeated data will be stored in the `.tmp/aggregate/data` directory.

This step can be skipped if you have already aggregated the data.

In [2]:
explanations = GradientBoostGlobalExplanations(
    data_folder='../../data/explanations/',
    model_name='TelcoGb_Model',
    to_date=datetime.date(2025,5,19)
)
explanations.generate_aggregations(overwrite=True)

## Simple Data Exploration
For quick overview of the data, you can use the ExplanationsExplorer class. It provides a simple interface to select a context and then plot the corrsponding predictor contribution plots.

In [12]:
explorer = ExplanationsExplorer()

### Selecting a context
Select the desired context from the list available on the right.
* Selecting 'Any' means the plots will display and aggegation across all contexts.
* Selecting a specific context will result in plots which aggregate the data for that context only.

If you have a very large list of possible contexts, you can filter the list on the right by selecting specific context keys from the comboboxes on the left.

In [13]:
explorer.context_selector()

GridBox(children=(HTML(value='<h3>Select Context Filters</>', layout=Layout(width='auto')), HTML(value='<h3>Se…

## Plotting the explanation data

The method shown below can be used to plot the explanation data for context selected above.
* The first plot shows the top 10 predictors sorted by their average contributions to predictions, the number can be changed by passing a different value to the `top_n` parameter.
* The subsequent plots show the avergae prediction contributions for the different values of top predictors.
    * Numeric predictor values are binned into deciles.
    * Categorical predictors are sorted and limited to the top 10 highest contributing values.

Note that the number of top predictors and top values can be changed by passing different values for the `top_n` and `top_k` parameters, respectively.

Additionally, if you are interested seeing the least contribution predictors, the `descending` parameter can be set to `False`. This will plot the least contributing predictors instead of the most contributing ones.

In [16]:
plots = explorer.plot_contributions(top_n = 5, top_k = 5)

## Advaced Data Exploration
For more advanced data explortation you can directly use the ExplanationsDataLoader and ExplanationsDataLoader classes. These classes provide more flexibility in how the data is loaded and processed. Allowing you to inspect the data before plotting.

In [2]:
loader = ExplanationsDataLoader(data_location='.tmp/aggregated_data')

In [20]:
top_predictors = loader.get_top_n_predictor_contribution_overall(top_n = 5, remaining=False)
top_predictors

partition,predictor_name,predictor_type,contribution,contribution_abs,contribution_weighted,contribution_weighted_abs,frequency,contribution_min,contribution_max
str,str,str,f64,f64,f64,f64,i64,f64,f64
"""whole_model""","""BILL_5""","""NUMERIC""",-0.001236,0.00124,-3.1498e-7,3.1680e-7,50000,-0.001995,0.000107
"""whole_model""","""MOD_1""","""NUMERIC""",-0.00096,0.00153,-6.1828e-7,8.3989e-7,50000,-0.002532,0.001556
"""whole_model""","""BILL_4""","""NUMERIC""",-0.00095,0.000981,-2.4136e-7,2.5235e-7,50000,-0.001872,0.000758
"""whole_model""","""BILL_3""","""NUMERIC""",-0.000881,0.000921,-2.2045e-7,2.3277e-7,50000,-0.001892,0.000695
"""whole_model""","""PRD_17""","""SYMBOLIC""",-0.000821,0.00105,-2.2651e-7,6.0550e-7,50000,-0.003538,0.001114


We can get the top predictors and insptect their most influential values

In [24]:
predictors = top_predictors.select(pl.col('predictor_name')).unique().to_series().to_list()
loader.get_top_k_predictor_value_contribution_overall(predictors=predictors, top_k = 5, remaining=False)

partition,predictor_name,predictor_type,bin_order,bin_contents,contribution,contribution_abs,contribution_weighted,contribution_weighted_abs,frequency,contribution_min,contribution_max,sort_column,sort_value
str,str,str,i64,str,f64,f64,f64,f64,i64,f64,f64,str,f64
"""whole_model""","""BILL_3""","""NUMERIC""",4,"""[7.520:12.180]""",-0.001174,0.001174,-0.000117,0.000117,5000,-0.001867,-0.000198,"""bin_order""",4.0
"""whole_model""","""BILL_3""","""NUMERIC""",5,"""[12.180:16.470]""",-0.001228,0.001228,-0.000123,0.000123,5000,-0.001798,-0.000484,"""bin_order""",5.0
"""whole_model""","""BILL_3""","""NUMERIC""",6,"""[16.470:19.950]""",-0.001219,0.001219,-0.000122,0.000122,5000,-0.001872,-0.000328,"""bin_order""",6.0
"""whole_model""","""BILL_3""","""NUMERIC""",7,"""[19.950:23.920]""",-0.001202,0.001202,-0.00012,0.00012,5000,-0.001847,-0.00036,"""bin_order""",7.0
"""whole_model""","""BILL_3""","""NUMERIC""",8,"""[23.920:29.635]""",-0.00118,0.00118,-0.000118,0.000118,5000,-0.001797,-0.000358,"""bin_order""",8.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""whole_model""","""MOD_1""","""NUMERIC""",7,"""[4.000:5.000]""",-0.002099,0.002099,-0.00063,0.00063,15000,-0.00252,-0.00124,"""bin_order""",7.0
"""whole_model""","""MOD_1""","""NUMERIC""",10,""">5.000""",-0.001991,0.001991,-0.000199,0.000199,5000,-0.002532,-0.001239,"""bin_order""",10.0
"""whole_model""","""PRD_17""","""SYMBOLIC""",1,"""V1""",0.000336,0.000345,0.000226,0.000232,33573,-0.000153,0.001114,"""contribution""",0.000336


Let's repeat the same again, but this time we will inspect a specific context, pyName = P64, instead of the entire model.

In [26]:
context_info = loader.get_context_infos()[0]
context_info['pyName'] = 'P64'
top_predictors_for_P64 = loader.get_top_n_predictor_contribution_by_context(context=context_info, top_n=5, remaining=False)
top_predictors_for_P64


partition,predictor_name,predictor_type,contribution,contribution_abs,contribution_weighted,contribution_weighted_abs,frequency,contribution_min,contribution_max
str,str,str,f64,f64,f64,f64,i64,f64,f64
"""{""partition"":{""pyChannel"":""Peg…","""BILL_5""","""NUMERIC""",-0.001219,0.001223,-3.0974e-7,3.1130e-7,571,-0.001936,0.000106
"""{""partition"":{""pyChannel"":""Peg…","""MOD_1""","""NUMERIC""",-0.001213,0.00167,-6.0369e-7,8.3190e-7,571,-0.002471,0.0013
"""{""partition"":{""pyChannel"":""Peg…","""BILL_4""","""NUMERIC""",-0.00105,0.001065,-2.6674e-7,2.7771e-7,571,-0.001854,0.000288
"""{""partition"":{""pyChannel"":""Peg…","""BILL_2""","""NUMERIC""",-0.000866,0.00095,-2.0416e-7,2.4569e-7,571,-0.001475,0.00088
"""{""partition"":{""pyChannel"":""Peg…","""BILL_3""","""NUMERIC""",-0.000865,0.000913,-2.1503e-7,2.2969e-7,571,-0.001799,0.000536


In [27]:
predictors_for_P64 = top_predictors_for_P64.select(pl.col('predictor_name')).unique().to_series().to_list()
loader.get_top_k_predictor_value_contribution_by_context(predictors=predictors_for_P64, top_k=5, context=context_info, remaining=False)

partition,predictor_name,predictor_type,bin_order,bin_contents,contribution,contribution_abs,contribution_weighted,contribution_weighted_abs,frequency,contribution_min,contribution_max,sort_column,sort_value
str,str,str,i64,str,f64,f64,f64,f64,i64,f64,f64,str,f64
"""{""partition"":{""pyChannel"":""Peg…","""BILL_2""","""NUMERIC""",5,"""[14.710:18.395]""",-0.001096,0.001096,-0.000109,0.000109,57,-0.001386,-0.000221,"""bin_order""",5.0
"""{""partition"":{""pyChannel"":""Peg…","""BILL_2""","""NUMERIC""",6,"""[18.395:20.590]""",-0.001138,0.001138,-0.000114,0.000114,57,-0.001438,-0.000257,"""bin_order""",6.0
"""{""partition"":{""pyChannel"":""Peg…","""BILL_2""","""NUMERIC""",7,"""[20.590:23.530]""",-0.001146,0.001146,-0.000114,0.000114,57,-0.001475,-0.000403,"""bin_order""",7.0
"""{""partition"":{""pyChannel"":""Peg…","""BILL_2""","""NUMERIC""",8,"""[23.530:29.410]""",-0.001157,0.001157,-0.000116,0.000116,57,-0.001428,-0.000621,"""bin_order""",8.0
"""{""partition"":{""pyChannel"":""Peg…","""BILL_2""","""NUMERIC""",9,"""[29.410:38.640]""",-0.001083,0.001083,-0.000108,0.000108,57,-0.001416,-0.000614,"""bin_order""",9.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""{""partition"":{""pyChannel"":""Peg…","""MOD_1""","""NUMERIC""",1,"""<=2.000""",0.000812,0.000812,0.000164,0.000164,115,0.000339,0.0013,"""bin_order""",1.0
"""{""partition"":{""pyChannel"":""Peg…","""MOD_1""","""NUMERIC""",3,"""[2.000:3.000]""",-0.000623,0.001287,-0.000124,0.000257,114,-0.002445,0.000957,"""bin_order""",3.0
"""{""partition"":{""pyChannel"":""Peg…","""MOD_1""","""NUMERIC""",5,"""[3.000:4.000]""",-0.002147,0.002147,-0.000214,0.000214,57,-0.002465,-0.001767,"""bin_order""",5.0
